In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import requests
import numpy as np
import tensorflow as tf
from bs4 import BeautifulSoup


In [ ]:

# Exemplo de URL de onde coletar dados
url = "https://pt.wikipedia.org/wiki/Transtornos_do_espectro_autista"

# Enviar requisição HTTP para a URL
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Coletar parágrafos de texto da página
paragraphs = soup.find_all('p')
text_data = [para.get_text() for para in paragraphs]

# Juntar os parágrafos em um único texto
text = ' '.join(text_data)

# Salvar o texto coletado em um arquivo
with open('arquivoTEA.txt', 'w', encoding='utf-8') as file:
    file.write(text)

with open('arquivoTEA.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Pré-processamento do texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Criar sequências de entrada e saída
input_sequences = []
for line in text.split('. '):  # Dividir o texto em sentenças
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Padronizar as sequências
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Criar inputs e labels
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Treinamento do modelo
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(total_words, activation='softmax'))

# Compilação do modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinamento do modelo
model.fit(xs, ys, epochs=10, verbose=1)

def responder_pergunta(model, tokenizer, pergunta, max_sequence_len, num_palavras=20):
    seed_text = pergunta
    for _ in range(num_palavras):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break

        seed_text += " " + output_word
        if output_word == "":  # Interrompe se não conseguir prever uma palavra
            break

    return seed_text

# Exemplo de uso
pergunta = input("Faça uma pergunta: ")
resposta = responder_pergunta(model, tokenizer, pergunta, max_sequence_len)
print(resposta)


Epoch 1/10
130/130 [==============================] - 157s 1s/step - loss: 6.8079 - accuracy: 0.0492
Epoch 2/10
130/130 [==============================] - 151s 1s/step - loss: 6.3822 - accuracy: 0.0535
Epoch 3/10
  6/130 [>.............................] - ETA: 2:44 - loss: 6.3413 - accuracy: 0.0573